# Multivariate Calculus
## Functions of two variables
Solving for two variables will be very similar to how we solved things in the previous week. Instead of a single variable however we will be using multiple. Suppose we have the following equation:

f(x, y) = 4x^2 - 9xy +6y^3

Let us find partial derivatives for x and y individually. It is incredibly easy with sympy as we saw last week.

In [99]:
from sympy import symbols

x, y = symbols('x, y')
f_x_y = 4 * x**2 - 9 * x * y + 6 * y**3

partial_f_x = f_x_y.diff(x)  # Differentiate our equation with respect to x
print(f"Partial Derivitive of our equation with respect to x: {partial_f_x}")

partial_f_y = f_x_y.diff(y)  # Differentiate our equation with respect to y
print(f"Partial Derivitive of our equation with respect to y: {partial_f_y}")

Partial Derivitive of our equation with respect to x: 8*x - 9*y
Partial Derivitive of our equation with respect to y: -9*x + 18*y**2


We can continue by taking the second derivitive of either. 

In [30]:
partial_y_y = partial_f_y.diff(y) 
partial_y_x = partial_f_y.diff(x)

print(f"Second derivitive fyy(x,y): {partial_y_y}")
print(f"Second derivitive fyx(x,y): {partial_y_x}")

36*y
Second derivitive fyy(x,y): 36*y
Second derivitive fyx(x,y): -9


## Finding Extrema
We can also find extrema with multiple variables in the same way we found them last week: use sympy's solve or solveset to get our answer. Suppose we have the equation:

f(x, y) = 6*x^2 + 6 * y^2 + 6 * x * y + 36 * x  - 5

Let us find all critical points for this equation. We begin by getting the partials for x and y. This gives us a system of linear equations. We can use sympy's linsolve function to solve this system of equations for x and y values respectively. 

In [35]:
from sympy import symbols, S, linsolve

x, y = symbols('x, y')

f_x_y = 6*x**2 + 6*y**2 + 6*x*y + 36*x - 5
partial_x = f_x_y.diff(x)
partial_y = f_x_y.diff(y)

results = linsolve([partial_x, partial_y], x, y)

print(f"The x and y value for critical points are {results}")


The x and y value for critical points are FiniteSet((-4, 2))


We now want to see if the critical point we've found is a relative maxima, minima, saddle point, or none of them. We do this by calculating the discriminant D. To help do this, let us create a helper function that implements the discremint formula and use it on our results of (-4, 2)

In [101]:
from sympy import lambdify, Abs

def D(func, x_sym, y_sym, x_crit, y_crit):
    # Calculate the discriminant for a given function
    f_x_x = func.diff(x_sym, x_sym)
    f_y_y = func.diff(y_sym, y_sym)
    f_x_y = func.diff(x_sym, y_sym)
    
    # Create callable functions for each of the derivatives we created
    lambd_x_x = lambdify([x_sym, y_sym], f_x_x)
    lambd_y_y = lambdify([x_sym, y_sym], f_y_y)
    lambd_x_y = lambdify([x_sym, y_sym], f_x_y)

    fxx_ab = lambd_x_x(x_crit, y_crit)
    fyy_ab = lambd_y_y(x_crit, y_crit) 
    fxy_ab = lambd_x_y(x_crit, y_crit)

    d = fxx_ab * fyy_ab - Abs(fxy_ab)**2

    print(f"The discriminant of our critical value is {d}")

    if d < 0:
        print("This is a saddle point")
    elif d > 0:
        if fxx_ab < 0:
            print("This is relative maxima")
        else:
            print("This is a relative minima")
    

# Call our new function and pass in the values
D(f_x_y, x, y, -4, 2)

The discriminant of our critical value is -369
This is a saddle point


Let us try another equation that has multiple points to test. We will use the nonlinsolve this time as the system of equations is nonlinear. You will notice that one of the solutions given includes non-real numbers. We use the is_real parameter to ignore that solution.

f(x, y) = 9xy - x^3 - y^3 - 6

In [102]:
from sympy import symbols, S, nonlinsolve

x, y = symbols('x, y', real=True)

f_x_y = 9*x*y - x**3 - y**3 - 6
partial_x = f_x_y.diff(x)
partial_y = f_x_y.diff(y)

# Get nonlinear solution and remove imaginary numbers
results = nonlinsolve([partial_x, partial_y], [x, y])

print(f"The x and y value for critical points are {results}")

for result in list(results):
    if result[0].is_real and result[1].is_real:  # Ignore any solutions that are not real numbers
        print(f"Analyzing critical point {result}")
        D(f_x_y, x, y, result[0], result[1])  

The x and y value for critical points are FiniteSet((0, 0), (3, 3), ((-3/2 - 3*sqrt(3)*I/2)**2/3, -3/2 - 3*sqrt(3)*I/2), ((-3/2 + 3*sqrt(3)*I/2)**2/3, -3/2 + 3*sqrt(3)*I/2))
Analyzing critical point (0, 0)
The discriminant of our critical value is -81
This is a saddle point
Analyzing critical point (3, 3)
The discriminant of our critical value is 243
This is relative maxima


## Lagrange Multipliers
When solving problems with greater dimensionality, it is helpful to use Lagrange multipliers. In python we will create an extra symbol l to represent our new variable lambda. The rest of the process is similar to that above.

Suppose we want to maximize the area of a rectangle x/y with the function f(x, y) = x *y

This is constrained however by the function:

g(x, y) = xy + 20y + 20x + 474000 = 500000

We first set our constraint equal to zero, and then solve for x and y.


In [103]:
from sympy import symbols, nonlinsolve, nsimplify, expand, powsimp

x, y, l = symbols('x, y, l', real=True)

f_xy = x * y
g_xy = x * y + 20 * y + 20 * x + 474000 - 500000  # Constraint is equal to zero when subtracting right side

# Implement lagrange function and solve system
f_xyl = f_xy - l *g_xy
f_xyl_x = f_xyl.diff(x)
f_xyl_y = f_xyl.diff(y)
f_xyl_l = f_xyl.diff(l)

results = nonlinsolve([
    f_xyl_x,
    f_xyl_y,
    f_xyl_l,
], [x, y, l])

# Analyze each of the solutions
for sol in list(results):
    x_sol, y_sol, l_sol = sol

    x_sol = x_sol.evalf()  # Evaluate the value so we're not getting sqrt equation
    y_sol = y_sol.evalf()
    print(f"Max value of x is {x_sol}")
    print(f"Max value of y is {y_sol}")

    print(f"Max area is of ({x_sol}, {y_sol}) is {x_sol * y_sol}")

Max value of x is 142.480768092719
Max value of y is 142.480768092719
Max area is of (142.480768092719, 142.480768092719) is 20300.7692762912
Max value of x is -182.480768092719
Max value of y is -182.480768092719
Max area is of (-182.480768092719, -182.480768092719) is 33299.2307237088


You can see that we were given two solutions for (x, y) that maximize the area  with the given constraint. The first solution of (142.5, 142.5) seems to have a lower max area than our second solution. The second solution however uses negative values for x and y which are not possible when calculating the area of a rectangle. 